In [1]:
# common packages 
import numpy as np 
import os
import copy
from math import *
import matplotlib.pyplot as plt
from functools import reduce
# reading in dicom files
import pydicom
# skimage image processing packages
from skimage import measure, morphology
#from skimage.morphology impo190rt ball, binary_closing
from skimage.measure import label, regionprops
# scipy linear algebra functions 
from scipy.linalg import norm
import scipy.ndimage
# ipywidgets for some interactive plots
from ipywidgets.widgets import * 
import ipywidgets as widgets
# plotly 3D interactive graphs 
import plotly
from plotly.graph_objs import *
#import chart_studio.plotly as py
# set plotly credentials here 
# this allows you to send results to your account plotly.tools.set_credentials_file(username=your_username, api_key=your_key)
import nrrd
import cv2

In [31]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input

model = InceptionResNetV2(weights='imagenet', include_top=False)

pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"

def maskcroppingbox(images_array, use2D=False):
    images_array_2 = np.argwhere(images_array)
    
    (zstart, ystart, xstart), (zstop, ystop, xstop) = images_array_2.min(axis=0), images_array_2.max(axis=0) + 1
    return (zstart, ystart, xstart), (zstop, ystop, xstop)
        
def featureextraction(image_array,mask_array):

    (zstart, ystart, xstart), (zstop, ystop, xstop) = maskcroppingbox(mask_array, use2D=False)
    roi_images = image_array[zstart-1:zstop+1,ystart:ystop,xstart:xstop].transpose((2,1,0))
    roi_images1 = zoom(roi_images, zoom=[224/roi_images.shape[0], 224/roi_images.shape[1],1], order=3)
    roi_images2 = np.array(roi_images1,dtype=float)    


    x = image.img_to_array(roi_images2)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    x = np.transpose(x, (3, 1, 2, 0))
    x = np.repeat(x, 3, axis=-1)

    base_model_pool_features = model.predict(x)
    
    feature_map = base_model_pool_features[0]
    feature_map = feature_map.transpose((2,1,0))
    features = np.max(feature_map,-1)
    features = np.max(features,-1)
    deeplearningfeatures = collections.OrderedDict()
    for ind_,f_ in enumerate(features):
    	deeplearningfeatures[str(ind_)] = f_
         
    return deeplearningfeatures



featureDict = {}
for s in os.listdir(pathdicom):
    print(s)
    filename = os.path.join(pathdicom, s)

        
    for t in os.listdir(filename):

        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):

        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')

    
    deeplearningfeatures = featureextraction(readdatadicom,readdatanrrd) 

    result = deeplearningfeatures
    key = list(result.keys())
    key = key[0:]
        
    feature = []
    for jind in range(len(key)):
        feature.append(result[key[jind]])
        
    featureDict[s] = feature
    dictkey = key
    print(s)
    
dataframe = pd.DataFrame.from_dict(featureDict, orient='index', columns=dictkey)
dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\tabella_risultati.csv')
    



100
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 4s/step
100
101
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 395ms/step
101
102
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 643ms/step
102
103
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 457ms/step
103
104
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 467ms/step
104
105
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 276ms/step
105
106
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
106
107
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 416ms/step
107
108
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 906ms/step
108
109
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 346ms/step
109
110
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 519ms/step
110
111
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 983ms/step
111
112
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 468ms/step
112
113
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 904ms/step
113
114
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 252ms/step
114
115
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 401ms/step
115
116
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 336ms/step
116
117
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step
117
118
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 453ms/step
118
119
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 278ms/step
119
12
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 233ms/step
12
120
2/2 ━━━━━━━━━━━━━